In [1]:
print("hii lstm")

hii lstm


In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df=df.dropna()

In [6]:
X=df.drop('label',axis=1)

In [7]:
## Get the Dependent features
y=df['label']

In [8]:
import tensorflow as tf

In [9]:
tf.__version__

'2.9.2'

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
voc_size=5000

In [12]:
messages=X.copy()

In [15]:
messages['title'][2]

'Why the Truth Might Get You Fired'

In [14]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3235, 2864, 338, 4374, 540, 2242, 2565, 3550, 41, 2535],
 [3349, 4801, 2233, 3723, 4806, 3268, 833],
 [4530, 3698, 632, 381],
 [644, 2474, 4289, 1746, 2297, 2162],
 [3447, 4806, 4809, 4075, 2754, 4884, 4806, 216, 3329, 1462],
 [514,
  4582,
  4389,
  2823,
  2507,
  1332,
  4757,
  3526,
  445,
  4948,
  2414,
  2521,
  4630,
  1303,
  833],
 [1702, 1857, 2710, 1708, 1265, 1660, 3586, 594, 2365, 50, 2962],
 [1790, 188, 490, 826, 4615, 1726, 1332, 3614, 2365, 50, 2962],
 [839, 358, 2107, 3960, 4797, 4756, 3964, 1984, 1332, 4546],
 [2512, 1311, 2042, 1344, 1797, 938, 4060, 3455],
 [3621, 4390, 2910, 4326, 2547, 1669, 2766, 835, 2740, 4694, 4856],
 [1746, 4436, 540, 4756, 1332, 4615],
 [2322, 4740, 4299, 205, 4158, 2040, 2077, 389, 2791],
 [719, 3631, 3884, 1166, 4628, 2421, 3147, 2365, 50, 2962],
 [1740, 3711, 1933, 361, 4518, 2365, 50, 2962],
 [4029, 4459, 1674, 726, 4508, 1146, 1066, 2483, 646, 3677],
 [1154, 2796, 4801],
 [312, 460, 3117, 3217, 1332, 3273, 3536, 833],
 [2205, 2006, 

In [20]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3550   41 2535]
 [   0    0    0 ... 4806 3268  833]
 [   0    0    0 ... 3698  632  381]
 ...
 [   0    0    0 ... 2365   50 2962]
 [   0    0    0 ...   72 4690  945]
 [   0    0    0 ...  354 1505 2363]]


In [21]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3235,
       2864,  338, 4374,  540, 2242, 2565, 3550,   41, 2535], dtype=int32)

In [22]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
len(embedded_docs),y.shape

(18285, (18285,))

In [24]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [26]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 42ms/step - loss: 0.3315 - accuracy: 0.8438 - val_loss: 0.2091 - val_accuracy: 0.9145
Epoch 2/10
192/192 [==============================] - 8s 43ms/step - loss: 0.1399 - accuracy: 0.9456 - val_loss: 0.1943 - val_accuracy: 0.9220
Epoch 3/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0957 - accuracy: 0.9630 - val_loss: 0.2431 - val_accuracy: 0.9203
Epoch 4/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0673 - accuracy: 0.9774 - val_loss: 0.3011 - val_accuracy: 0.9181
Epoch 5/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0465 - accuracy: 0.9859 - val_loss: 0.2992 - val_accuracy: 0.9132
Epoch 6/10
192/192 [==============================] - 13s 67ms/step - loss: 0.0309 - accuracy: 0.9901 - val_loss: 0.3619 - val_accuracy: 0.9095
Epoch 7/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0209 - accuracy: 0.9944 - val_loss: 0.3688 - val_accuracy: 0.91

In [27]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [29]:
y_pred=model.predict(X_test)

189/189 [==============================] - 3s 10ms/step


In [32]:
y_pred

array([[0.498749  ],
       [0.49832484],
       [0.4991067 ],
       ...,
       [0.4998035 ],
       [0.49910364],
       [0.49928507]], dtype=float32)

In [33]:
y_test

array([1, 0, 0, ..., 0, 1, 1])

In [30]:
from sklearn.metrics import confusion_matrix

In [34]:
#confusion_matrix(y_test,y_pred)

In [36]:
#from sklearn.metrics import accuracy_score
#accuracy_score(y_test,y_pred)